In [1]:
import random
from time import time

import torch.nn as nn
from sklearn.metrics import f1_score

from config import cfg
from utils import AverageMeter
from utils import utilities

SEED = 2021
random.seed(SEED)

import torch

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.empty_cache()
import numpy as np

np.random.seed(SEED)
import torchvision.transforms as T
from torch.utils.data import DataLoader
from data.dataset import CIFAR100

batch_size = cfg.SOLVER.IMS_PER_BATCH
num_workers = cfg.DATALOADER.NUM_WORKERS
from RandAugment import RandAugment

## Train

In [2]:
normalize_transform = T.Normalize(mean=(0.5071, 0.4865, 0.4409),
                                  std=(0.2673, 0.2564, 0.2762))

transformer = T.Compose([
    T.RandomCrop(32, padding=4, padding_mode='reflect'),
    T.RandomHorizontalFlip(),
    # RandAugment(n=2, m=9),
    T.ToTensor(),
    normalize_transform
])
val_transform = T.Compose([
    T.ToTensor(),
    normalize_transform
])

In [3]:
train_dataset = CIFAR100(
    root='./data/dataset/cifar_100_dataset_file', train=True,
    download=True, transform=transformer,
)

validation_dataset = CIFAR100(
    root='./data/dataset/cifar_100_dataset_file', train=False,
    download=True, transform=val_transform,
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
train_data_loader = DataLoader(
    train_dataset, batch_size=batch_size, num_workers=num_workers
)

val_data_loader = DataLoader(
    validation_dataset, batch_size=cfg.TEST.IMS_PER_BATCH,
    num_workers=num_workers
)

In [13]:

def train(
        cfg,
        model,
        train_loader,
        val_loader,
        optimizer,
        scheduler,
        criterion,
        grad_clip=None
):
    model_name = cfg.MODEL.NAME
    log_period = cfg.SOLVER.LOG_PERIOD
    checkpoint_period = cfg.SOLVER.CHECKPOINT_PERIOD
    output_dir = cfg.DIR.OUTPUT_DIR
    device = cfg.MODEL.DEVICE
    device = torch.device(device=device)
    epochs = cfg.SOLVER.MAX_EPOCHS

    model = model.to(device)

    min_valid_loss = np.inf

    for e in range(epochs):
        batch_time = AverageMeter('Time', ':6.3f')
        data_time = AverageMeter('Data', ':6.3f')
        train_losses = AverageMeter('Training Loss', ':.4e')
        val_losses = AverageMeter('Validation Loss', ':.4e')
        train_accuracy = AverageMeter('Training Accuracy', ':6.2f')
        val_accuracy = AverageMeter('Validation Accuracy', ':6.2f')
        train_f1 = AverageMeter('Training F1 score', ':6.2f')
        val_f1 = AverageMeter('Validation F1 score', ':6.2f')

        start_epoch = time()
        model.train()
        for itr, (data, labels) in enumerate(train_loader):
            start_batch = time()
            # Transfer Data to GPU if available
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()


            # Forward Pass
            target = model(data)
            # Find the Loss
            loss = criterion(target, labels)
            # Calculate gradients
            loss.backward()

             # Gradient clipping
            if grad_clip:
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            # Update Weights
            optimizer.step()
              # Clear the gradients
            optimizer.zero_grad()
            if scheduler:
                scheduler.step()
            # Calculate Loss
            # train_loss = loss.item() * data.size(0)
            # accuracy
            acc = utilities.accuracy(y_true=labels, y_pred=target)
            # print(acc)
            _, predicted = torch.max(target.data, 1)
            f = f1_score(labels.cpu(), predicted.cpu(), average='micro')
            # print(f)
            train_f1.update(f)
            train_accuracy.update(acc)
            train_losses.update(loss.item(), data.size(0))
            # measure elapsed time
            batch_time.update(time() - start_batch)

            utilities.progress_bar(current=itr, total=len(train_loader))

        if val_loader:
            model.eval()  # Optional when not using Model Specific layer
            for data, labels in val_loader:
                # Transfer Data to GPU if available
                if torch.cuda.is_available():
                    data, labels = data.cuda(), labels.cuda()

                # Forward Pass
                target = model(data)
                # Find the Loss
                loss = criterion(target, labels)
                # Calculate Loss
                # valid_loss = loss.item() * data.size(0)
                val_losses.update(loss.item(), data.size(0))

                acc = utilities.accuracy(y_true=labels, y_pred=target)
                # print(f"acc{acc}")
                # accuracy
                _, predicted = torch.max(target.data, 1)
                val_accuracy.update(acc)
                f = f1_score(labels.cpu(), predicted.cpu(), average='micro')
                # print(f"f{f}")
                val_f1.update(f)

        data_time.update(time() - start_epoch)

        print(
            f'Epoch {e + 1} [{data_time.avg:.2f}s]: Training Loss: {train_losses.avg:.2f} '
            f'Validation Loss: {val_losses.avg:.2f}, Train Accuracy: {train_accuracy.avg:.2f}, Train f1 score: {train_f1.avg:.2f},'
            f'Validation Accuracy: {val_accuracy.avg:.2f}, Validation f1 score: {val_f1.avg:.2f}')

        # if min_valid_loss > val_losses.avg:
        #     print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_losses.avg:.6f}) \t Saving The Model')
        #     min_valid_loss = val_losses.avg
        #
        #     # Saving State Dict
        #     torch.save(model.state_dict(), cfg.DIR.BEST_MODEL + cfg.TEST.WEIGHT)

    print('Finished Training')

In [6]:
# len(train_data_loader) * epochs

In [8]:
from torchvision.models.resnet import resnet18 as _resnet18
import torch.optim as optim


max_lr = 0.01
grad_clip = 0.1
weight_decay = 1e-4

model = _resnet18(pretrained=False)
# Set up cutom optimizer with weight decay
optimizer = optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)
# Set up one-cycle learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=cfg.SOLVER.IMS_PER_BATCH,
                                            steps_per_epoch=len(train_data_loader))
criterion = nn.CrossEntropyLoss()

In [9]:
# train(cfg=cfg, model=model,
#       optimizer=optimizer, scheduler=sched, train_loader=train_data_loader,
#       val_loader=val_data_loader, criterion=criterion)

Epoch 1 [14.88s]: Training Loss: 3.92 Validation Loss: 3.34, Train Accuracy: 0.13, Train f1 score: 0.13,Validation Accuracy: 0.20, Validation f1 score: 0.20
Epoch 2 [14.83s]: Training Loss: 3.21 Validation Loss: 3.00, Train Accuracy: 0.22, Train f1 score: 0.22,Validation Accuracy: 0.26, Validation f1 score: 0.26
Epoch 3 [14.88s]: Training Loss: 2.91 Validation Loss: 2.82, Train Accuracy: 0.27, Train f1 score: 0.27,Validation Accuracy: 0.29, Validation f1 score: 0.29
Epoch 4 [14.98s]: Training Loss: 2.70 Validation Loss: 2.60, Train Accuracy: 0.31, Train f1 score: 0.31,Validation Accuracy: 0.34, Validation f1 score: 0.34
Epoch 5 [15.00s]: Training Loss: 2.52 Validation Loss: 2.51, Train Accuracy: 0.35, Train f1 score: 0.35,Validation Accuracy: 0.36, Validation f1 score: 0.36
Epoch 6 [15.10s]: Training Loss: 2.39 Validation Loss: 2.39, Train Accuracy: 0.38, Train f1 score: 0.38,Validation Accuracy: 0.38, Validation f1 score: 0.38
Epoch 7 [15.11s]: Training Loss: 2.27 Validation Loss: 2.3

In [10]:
from numpy import argmax
from numpy import vstack
from sklearn.metrics import accuracy_score
# evaluate the model
def evaluate_model(test_dl, model):
  predictions, actuals = list(), list()
  for i, (inputs, targets) in enumerate(test_dl):
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()
    # evaluate the model on the test set
    yhat = model(inputs)
    # retrieve numpy array
    yhat = yhat.detach().cpu().numpy()
    actual = targets.cpu().numpy()
    # convert to class labels
    yhat = argmax(yhat, axis=1)
    # reshape for stacking
    actual = actual.reshape((len(actual), 1))
    yhat = yhat.reshape((len(yhat), 1))
    # store
    predictions.append(yhat)
    actuals.append(actual)
  predictions, actuals = vstack(predictions), vstack(actuals)
  # calculate accuracy
  acc = accuracy_score(actuals, predictions)
  return acc

acc = evaluate_model(test_dl=val_data_loader,model=model)
print(acc)

0.4808


In [11]:
from torchvision.models.resnet import resnet18 as _resnet18
import torch.optim as optim


max_lr = 0.001
grad_clip = 0.1
weight_decay = 1e-4

model2 = _resnet18(pretrained=False)
# Set up cutom optimizer with weight decay
optimizer2 = optim.Adam(model.parameters(), max_lr, weight_decay=weight_decay)
# Set up one-cycle learning rate scheduler
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer2, max_lr, epochs=cfg.SOLVER.IMS_PER_BATCH,
                                            steps_per_epoch=len(train_data_loader))
criterion = nn.CrossEntropyLoss()

In [15]:
model2 = train(cfg=cfg, model=model2,
      optimizer=optimizer2, scheduler=sched, train_loader=train_data_loader,
      val_loader=val_data_loader, criterion=criterion)

Epoch 1 [14.97s]: Training Loss: 7.24 Validation Loss: 7.24, Train Accuracy: 0.00, Train f1 score: 0.00,Validation Accuracy: 0.00, Validation f1 score: 0.00
Epoch 2 [14.92s]: Training Loss: 7.24 Validation Loss: 7.24, Train Accuracy: 0.00, Train f1 score: 0.00,Validation Accuracy: 0.00, Validation f1 score: 0.00
Epoch 3 [15.04s]: Training Loss: 7.24 Validation Loss: 7.24, Train Accuracy: 0.00, Train f1 score: 0.00,Validation Accuracy: 0.00, Validation f1 score: 0.00
Epoch 4 [15.10s]: Training Loss: 7.23 Validation Loss: 7.24, Train Accuracy: 0.00, Train f1 score: 0.00,Validation Accuracy: 0.00, Validation f1 score: 0.00


Traceback (most recent call last):
  File "/home/omid/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/omid/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/omid/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/omid/anaconda3/envs/pytorch/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


KeyboardInterrupt: 